# Derivados Financieros 2022 - Clase 1
## Limpieza del panel de opciones

### Importo los modulos

In [1]:
import sys
sys.path.append('..')

from Codigo.data_byma import *
from Codigo.utils_opciones_byma import *



##Del panel de Iol crudo a otro con informacion customizada

### Metodo para generar el panel customizado

a) Panel crudo mas informacion masticada (columnas de la derecha)

In [ ]:
panel_opciones_byma_all_raw = obtener_panel_opciones_byma(ticker = '', clean_flag = False)
panel_opciones_byma_all_raw

AttributeError: 'NoneType' object has no attribute 'find_all'

b) Panel limpiotodos los tickers

In [ ]:
panel_opciones_byma_all_clean = obtener_panel_opciones_byma(ticker = '', clean_flag = True)
panel_opciones_byma_all_clean

AttributeError: 'NoneType' object has no attribute 'find_all'

b) Panel limpio paa un ticker en particular

In [ ]:
panel_opciones_byma_GGAL_clean = obtener_panel_opciones_byma('GGAL', True)
panel_opciones_byma_GGAL_clean

### Que hace el metodo internamente?



#### 1) Levanta los paneles de opciones y de acciones y genera una copia del de opciones

In [23]:
panel_iol = web_scraping_opciones_iol()
#Obtengo el panel crudo de acciones (para el spot)
panel_acciones = web_scraping_acciones_iol()
panel_acciones
#Genero el nuevo panel
panel_opciones = panel_iol.copy()


In [24]:


panel_opciones = panel_opciones.rename(columns={'Símbolo': 'Especie', 'ÚltimoCierre': 'Last', 'ÚltimoOperado': 'Last'}) 
panel_opciones

,Especie,Last,Apertura,Mínimo,Máximo,Last,MontoOperado
0,ALUC10179O,51.00,51.00,51.00,51.00,51.00,102000.0
1,ALUC11179O,38.00,38.00,38.00,38.00,38.00,0.0
2,ALUC115.DI,47.00,48.00,47.00,48.00,47.00,0.0
3,ALUC12679O,31.00,31.00,30.00,31.00,31.00,9200.0
4,ALUC145.OC,14.00,14.00,11.75,14.00,14.00,82925.0
...,...,...,...,...,...,...,...
331,YPFV840.OC,10.27,11.00,8.08,11.00,10.27,0.0
332,YPFV880.OC,28.00,28.00,28.00,28.00,28.00,0.0
333,YPFV920.DI,8.75,8.75,8.75,8.75,8.75,0.0
334,YPFV920.OC,1.15,1.15,0.87,1.15,1.15,3009.0


#### 2) Inicializo las columnas nuevas a agregar del Dataframe

In [25]:
#Le agrego las nuevas columnas (ojo con los tipos string, float, int, datetime)
panel_opciones['Ticker_Opcion'] = ''
panel_opciones['Ticker_Stock'] = ''
panel_opciones['Tipo'] = ''
panel_opciones['Strike'] = 0.0
panel_opciones['ExpiryMonthName'] = ''
panel_opciones['ExpiryMonthNumber'] = 0
panel_opciones['ExpiryDate'] = datetime.date.today()
panel_opciones['Spot'] = 0.0
panel_opciones['TTM'] = 0
panel_opciones['Moneyness'] = 0.0

panel_opciones

,Especie,Last,Apertura,Mínimo,Máximo,Last,MontoOperado,Ticker_Opcion,Ticker_Stock,Tipo,Strike,ExpiryMonthName,ExpiryMonthNumber,ExpiryDate,Spot,TTM,Moneyness
0,ALUC10179O,51.00,51.00,51.00,51.00,51.00,102000.0,,,,0.0,,0,2022-08-24,0.0,0,0.0
1,ALUC11179O,38.00,38.00,38.00,38.00,38.00,0.0,,,,0.0,,0,2022-08-24,0.0,0,0.0
2,ALUC115.DI,47.00,48.00,47.00,48.00,47.00,0.0,,,,0.0,,0,2022-08-24,0.0,0,0.0
3,ALUC12679O,31.00,31.00,30.00,31.00,31.00,9200.0,,,,0.0,,0,2022-08-24,0.0,0,0.0
4,ALUC145.OC,14.00,14.00,11.75,14.00,14.00,82925.0,,,,0.0,,0,2022-08-24,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,YPFV840.OC,10.27,11.00,8.08,11.00,10.27,0.0,,,,0.0,,0,2022-08-24,0.0,0,0.0
332,YPFV880.OC,28.00,28.00,28.00,28.00,28.00,0.0,,,,0.0,,0,2022-08-24,0.0,0,0.0
333,YPFV920.DI,8.75,8.75,8.75,8.75,8.75,0.0,,,,0.0,,0,2022-08-24,0.0,0,0.0
334,YPFV920.OC,1.15,1.15,0.87,1.15,1.15,3009.0,,,,0.0,,0,2022-08-24,0.0,0,0.0


#### 3) Recorro fila por fila para llenar las nuevas columnas (1 ejemplo)

##### *) Desgloso la especie en ticker, strike, vencimiento y tipo

In [37]:


fila =0
panel_opciones.at[fila,'Ticker_Opcion']  = panel_opciones.Especie.values[fila][0:3]
panel_opciones.at[fila,'Ticker_Stock'] = conversor_ticker(panel_opciones.Ticker_Opcion.values[fila])
panel_opciones.at[fila,'Tipo'] = panel_opciones.Especie.values[fila][3:4]
panel_opciones.at[fila,'Tipo']  = panel_opciones.Tipo.values[fila].replace('V', 'P')

panel_opciones.head(3)


,Especie,Last,Apertura,Mínimo,Máximo,Last,MontoOperado,Ticker_Opcion,Ticker_Stock,Tipo,Strike,ExpiryMonthName,ExpiryMonthNumber,ExpiryDate,Spot,TTM,Moneyness
0,ALUC10179O,51.0,51.0,51.0,51.0,51.0,102000.0,ALU,ALUA,C,0.0,,0,2022-08-24,0.0,0,0.0
1,ALUC11179O,38.0,38.0,38.0,38.0,38.0,0.0,ALU,ALUA,C,0.0,,0,2022-08-24,0.0,0,0.0
2,ALUC115.DI,47.0,48.0,47.0,48.0,47.0,0.0,,,,0.0,,0,2022-08-24,0.0,0,0.0


##### *) Uno a uno voy generando lo necesario

In [40]:
# Artilugio que lo tuve que usar NO RECUERDO POR QUE! :)

try:
    int(panel_opciones.ExpiryMonthName.values[fila][0])
    panel_opciones.at[fila,'Strike'] = float(panel_opciones.Strike.values[fila] + int(panel_opciones.ExpiryMonthName.values[fila][0]))
    panel_opciones.at[fila,'ExpiryMonthName'] = panel_opciones.ExpiryMonthName.values[fila][1]
except:
    panel_opciones.at[fila,'Strike'] = float(panel_opciones.Strike.values[fila])
    
panel_opciones.head(2)  


,Especie,Last,Apertura,Mínimo,Máximo,Last,MontoOperado,Ticker_Opcion,Ticker_Stock,Tipo,Strike,ExpiryMonthName,ExpiryMonthNumber,ExpiryDate,Spot,TTM,Moneyness
0,ALUC10179O,51.0,51.0,51.0,51.0,51.0,102000.0,ALU,ALUA,C,0.0,,0,2022-08-24,0.0,0,0.0
1,ALUC11179O,38.0,38.0,38.0,38.0,38.0,0.0,ALU,ALUA,C,0.0,,0,2022-08-24,0.0,0,0.0


In [ ]:
#conversor_ticker('AGR')

#panel_opciones.Especie.values[fila][0:3]

#panel_opciones.Ticker_Opcion.values[fila]\

#conversor_ticker(panel_opciones.Ticker_Opcion.values[fila])

#panel_opciones.Ticker_Stock.values[fila]



##### *) Obtengo el precio spot del ticker con un metodo interno y aprovechando el panel de acciones

In [ ]:
panel_opciones.Spot.values[fila] = obtener_spot_ticker(panel_acciones, panel_opciones.Ticker_Stock.values[fila])

##### *) Obtengo el Time to maturity (en dias)

In [ ]:
try:
    panel_opciones.ExpiryMonthNumber.values[fila] = mes_nombre_a_numero(panel_opciones.ExpiryMonthName.values[fila])
except:
    panel_opciones.ExpiryMonthNumber.values[fila] = 0

panel_opciones.ExpiryDate.values[fila] = fecha_expiracion(panel_opciones.ExpiryMonthNumber.values[fila])
hoy = datetime.date.today()
panel_opciones.TTM.values[fila] = (panel_opciones.ExpiryDate.values[fila] - hoy).days

panel_opciones

##### *) Por ultimo calculo el moneyness de valor informativo importante

In [ ]:
panel_opciones.Moneyness.values[fila] = panel_opciones.Spot.values[fila] / panel_opciones.Strike.values[fila]

In [ ]:
panel_opciones.head(2)

#### 3bis) Recorro fila por fila para llenar las nuevas columnas


In [ ]:

for fila in range(1,len(panel_opciones.Especie.values)):
    panel_opciones.Ticker_Opcion.values[fila] = panel_opciones.Especie.values[fila][0:3]

    panel_opciones.Ticker_Stock.values[fila] = conversor_ticker(panel_opciones.Ticker_Opcion.values[fila])

    panel_opciones.Tipo.values[fila] = panel_opciones.Especie.values[fila][3:4]
    panel_opciones.Tipo.values[fila] = panel_opciones.Tipo.values[fila].replace('V', 'P')

    panel_opciones.Strike.values[fila] = panel_opciones.Especie.values[fila][4:8]

    panel_opciones.ExpiryMonthName.values[fila] = panel_opciones.Especie.values[fila][8:10]

    try:
        int(panel_opciones.ExpiryMonthName.values[fila][0])
        panel_opciones.Strike.values[fila] = float(panel_opciones.Strike.values[fila] + int(panel_opciones.ExpiryMonthName.values[fila][0]))
        panel_opciones.ExpiryMonthName.values[fila] = panel_opciones.ExpiryMonthName.values[fila][1]
    except:
        panel_opciones.Strike.values[fila] = float(panel_opciones.Strike.values[fila])

    try:
        panel_opciones.ExpiryMonthNumber.values[fila] = mes_nombre_a_numero(panel_opciones.ExpiryMonthName.values[fila])
    except:
        panel_opciones.ExpiryMonthNumber.values[fila] = 0

    panel_opciones.ExpiryDate.values[fila] = fecha_expiracion(panel_opciones.ExpiryMonthNumber.values[fila])


    panel_opciones.Spot.values[fila] = obtener_spot_ticker(panel_acciones, panel_opciones.Ticker_Stock.values[fila])


    hoy = datetime.date.today()
    panel_opciones.TTM.values[fila] = (panel_opciones.ExpiryDate.values[fila] - hoy).days

    panel_opciones.Moneyness.values[fila] = panel_opciones.Spot.values[fila] / panel_opciones.Strike.values[fila]



In [ ]:
#Asi queda entonces el panel
panel_opciones

#### 4) Filtro por ticker


In [ ]:
#Si se provee un ticker, se filtra por ticker, si no, se provee el panel completo

ticker = 'ALUA'

if ticker == '':
    pass
else:
    panel_opciones = panel_opciones[panel_opciones.Ticker_Stock == ticker]
    
panel_opciones

#### 5) Panel limpio

In [ ]:
panel_limpio = panel_opciones.copy()

clean_flag = True

# Panel limpio: cambio de nombre, reordenamiento y limpieza minima del panel
if clean_flag == True:
    # Sacar opciones sobre bonos
    panel_limpio = panel_limpio[~panel_limpio.Especie.str.contains('A24')]

    panel_limpio = panel_limpio[
            ['Especie', 'Ticker_Stock', 'Spot', 'Tipo', 'Strike', 'TTM', 'Último', 'Moneyness']]

    panel_limpio = panel_limpio.rename(
            columns={'Ticker_Stock': 'Ticker', 'Tipo': 'CallPut', 'Último': 'Last'}, inplace=False)

    panel_limpio = panel_limpio.sort_values(['TTM', 'Ticker', 'Strike', 'CallPut'])

    # Remover TTMs mayores de un año o negativos (por algun error)
    panel_limpio = panel_limpio[panel_limpio.TTM < 364]
    panel_limpio = panel_limpio[panel_limpio.TTM > 0]

    panel_limpio = panel_limpio.reset_index()
    del panel_limpio['index']
    
panel_limpio